In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler

import os
import re

Using TensorFlow backend.


In [2]:
EMBEDDING_FILES = [
    'embeddings/crawl-300d-2M.vec',
    'embeddings/glove.840B.300d.txt'
]

NUM_MODELS = 2
BATCH_SIZE = 128
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 2
MAX_LEN = 220

# Helper Functiosn

## Processing Embeddings

In [3]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

## Build and define the model

In [4]:
def build_model(embedding_matrix, model_name):
    words = Input(shape=(max_length,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.name = model_name

    return model

# Data Preprocessing

In [5]:
def load_directory_data(directory):
    data={}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

def load_dataset(directory):
    pos_df = load_directory_data(directory + "/pos")
    neg_df = load_directory_data(directory + "/neg")
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

In [6]:
train_df = load_dataset("data/aclImdb/train/")
test_df = load_dataset("data/aclImdb/test/")

In [7]:
x_train = train_df["sentence"]
y_train = train_df["sentiment"]

x_test = test_df["sentence"]
y_test = test_df["sentiment"]

In [8]:
from keras.utils import to_categorical

# y_train = to_categorical(train_df["sentiment"])
# y_test = to_categorical(test_df["sentiment"])

In [9]:
set(train_df['sentiment'].values), train_df["sentiment"][1], y_train[1]

({'1', '10', '2', '3', '4', '7', '8', '9'}, '3', '3')

In [10]:
x_train[1], y_train[1]

('Ursula Andress\' naked body is one of those things that make you believe in God. The other two women (especially the one who plays the maid) have great bodies as well. Then why is the higher grade that I can give to a film with such quality and quantity of nudity only 3 out of 10? Because, to get to Ursula\'s unbelievable body, we have to sit through a movie that is otherwise unfunny and boring (keep in mind that I watched the full 101-minute version, not the 78-minute American one which probably cuts out a lot of the extraneous material). In typical Italian-comedy tradition, most of the characters are exaggerated caricatures (the army freak, the "latin lover", the constant drunk, the naive maid) that are not funny, simply overacted. Final word: watch this, but keep your finger on the fast-forward button, you\'re gonna need it. (*)',
 '3')

## Tokenisation

In [11]:
tokenizer_obj = text.Tokenizer()
total_reviews = x_train + x_test
tokenizer_obj.fit_on_texts(total_reviews)

max_length = max([len(s.split()) for s in total_reviews])
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(x_test)

X_train_pad = sequence.pad_sequences(X_train_tokens, maxlen=max_length, padding="post")
X_test_pad = sequence.pad_sequences(X_test_tokens, maxlen=max_length, padding="post")

In [12]:
# embedding_matrix = np.concatenate(
#     [build_matrix(tokenizer_obj.word_index, f) for f in EMBEDDING_FILES], axis=-1)

# np.savetxt('embedding_concat.txt', embedding_matrix , fmt='%d')

In [13]:
embedding_matrix = np.loadtxt('embedding_concat.txt', dtype=int)

# Training Loop

In [14]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
LOG_DIR = '/media/eigenstir/1TBSecondary/tbgraphs'

tbCallBack = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model_mk1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
ls_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))

In [15]:
weights = []
for model_idx in range(NUM_MODELS):
    model = build_model(embedding_matrix, model_name = str(model_idx))
    for global_epoch in range(EPOCHS):
        model.fit(
            X_train_pad,
            y_train,
            validation_data=(X_test_pad, y_test),
            batch_size=BATCH_SIZE,
            epochs=1,
            verbose=2,
            callbacks=[tbCallBack, es, ModelCheckpoint('best_reg_model_mk1' + str(model_idx) + '.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)]
        )
        weights.append(2 ** global_epoch)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
 - 211s - loss: 32.1056 - val_loss: 32.5527
Train on 25000 samples, validate on 25000 samples


/home/eigenstir/.local/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 1/1
 - 220s - loss: 32.0660 - val_loss: 32.5527
Train on 25000 samples, validate on 25000 samples
Epoch 1/1


KeyboardInterrupt: 

# Possible Testing Texts

Testing texts: Uber

Neg:

1. #uber's customer service department is worst than comcast.
1. #Uber ...get your act right! # frustrated#waiting for competition
1. .@Uber charges more if they think you're willing to pay more. Uber thinks you're willing to pay more when your battery is about to die. And Uber knows when your battery is about to die.

Pos:
1. Ride-hailing industry expected to grow eightfold to $285 billion by 2030
1. Uber is awesome!
1. Best service ever with Uber


In [121]:
test_df.groupby('sentiment').count()

,sentence,polarity
sentiment,,
1,5022,5022
10,4999,4999
2,2302,2302
3,2541,2541
4,2635,2635
7,2307,2307
8,2850,2850
9,2344,2344


In [107]:
test_df["sentence"][1], test_df["sentiment"][1]

("I live in Mexico City, so I have to suffer throug the trailers for every piece of trash that comes out from all these stupid Mexican filmmakers. You want to admire a Mexican guy for making great films? Take a look at something by Guillermo del Toro (specially The Devil's Backbone), or maybe Alfonso Cuarón (though I really don't like his films, but I respect them).<br /><br />Mexican filmmakers often produce some of the most terrible utter trash ever (Por la Libre, El Segundo Aire, American Visa), but this is one of the lowest points in Mexican films ever. If you respect your brain, please avoid this piece of **** at all costs. It would be more intelligent to watch some video of a wedding or to watch Britney's reality show. That's got more IQ than everyone in this 'film'.",
 '1')

In [108]:
test_data = tokenizer_obj.texts_to_sequences(test_df["sentence"][1])
test_data = sequence.pad_sequences(test_data, maxlen=max_length, padding="post")

In [128]:
len(test_data[3]), len(test_df["sentence"][1])

(2574, 781)

In [133]:
prediction = model.predict(test_data)

In [110]:
len(prediction)

781

In [136]:
prediction[780]

array([1.4865126e-04, 2.0165390e-01, 8.5235134e-02, 6.8558104e-02,
       6.4780466e-02, 1.4810260e-04, 1.6418842e-04, 8.5876852e-02,
       1.5305586e-01, 1.0984342e-01, 3.5275525e-01], dtype=float32)

In [135]:
np.argmax(prediction[780].flatten())

10

In [139]:
imdb = tf.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 3s 0us/step


# Business Times Singapore

In [ ]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

# Backtrader


In [ ]:
import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)
        

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2018,11,25),
        reverse = False
    )
    
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

# Saving a model

In [ ]:
import sys
from keras.models import load_model
import tensorflow as tf
# from keras import backend as K
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants



In [ ]:
keras.backend.set_learning_phase(0)

model = build_model(embedding_matrix, model_name = str(model_idx))
config = model.get_config()
weights = model.get_weights()
new_model = Model.from_config(config)
new_model.set_weights(weights)

Export the model

In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

export_path = 'exported_model'
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'text': new_model.input},
                                  outputs={'sentiment': new_model.output})

with keras.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

# Another type of loop

In [15]:
embedding_matrix = np.loadtxt('embedding_concat.txt', dtype=int)
words = Input(shape=(max_length,))
x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
x = SpatialDropout1D(0.3)(x)
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
result = Dense(11, activation='sigmoid')(hidden)
    
model = Model(inputs=words, outputs=result)
model.compile(loss='binary_crossentropy', optimizer='adam')

IndentationError: unexpected indent (<ipython-input-15-c5479d416f9e>, line 3)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

tbCallBack = keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
ls_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))

In [ ]:
model.fit(X_train_pad, y_train, batch_size=64, epochs=10, 
          validation_data=(X_test_pad, y_test), callbacks=[tbCallBack, es, mc])